# Planetary Nebula Production <a class="tocSkip">
    
This notebook is used to test and showcase the results of my first project. I use spectroscopic data from the [Multi Unit Spectroscopic Explorer](https://www.eso.org/sci/facilities/develop/instruments/muse.html) (MUSE) that has been observed as part of the [PHANGS](https://sites.google.com/view/phangs/home) collaboration.
    
I will use a set of line maps of emission lines to identify Planetary Nebula in the data an measure their brightness. This can then be used to fit an empiric relation and hence measure the distance to the galaxy.
    
This notebook is used for developement. Final code is moved to the `pymuse` packge in the `src` folder. Any production scripts reside in the `scripts` folder.

## Preparation
 
### Load Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pymuse.packages import *

from pymuse.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

### Read in data

this uses the `ReadLineMaps` class from the `pymuse.io` module. To use it, we first need to specify the path to the data folder

IC5332, NGC628, NGC1087, NGC1365, NGC1512, NGC1566, NGC1672, NGC2835, NGC3351, NGC3627, NGC4254, NGC4535, NGC5068

In [ ]:
from pymuse.io import ReadLineMaps

name = 'NGC628'

# first we need to specify the path to the raw data
data_raw = Path('d:\downloads\MUSEDAP')
basedir = Path('..')

# list all files in the specified directory
galaxies = [x.name for x in data_raw.iterdir() if x.is_dir()]

extensions = ['OIII5006', 'HA6562', 'NII6583', 'SII6716']

# read in the data we will be working with and print some information
galaxy = ReadLineMaps(data_raw / name,extensions)

mask = np.zeros(galaxy.shape,dtype=bool)
mask |= galaxy.star_mask.astype(bool)

### Mask unwanted regions

In [ ]:
from pymuse.auxiliary import circular_mask
from pymuse.plot.plot import create_RGB
    
mask = np.zeros(galaxy.shape,dtype=bool)
mask |= galaxy.star_mask.astype(bool)

# define masks as slices
masks = {
 'NGC1365' : circular_mask(*galaxy.shape,(720,420),radius=210),
 'NGC1512' : circular_mask(*galaxy.shape,radius=80),
 'NGC1566' : circular_mask(*galaxy.shape,(450,450),radius=100),
 'NGC1672' : circular_mask(*galaxy.shape,(600,310),radius=100),
 'NGC3627' : circular_mask(*galaxy.shape,(330,740),radius=100),
 'NGC3351' : circular_mask(*galaxy.shape,radius=200),
 'NGC4535' : circular_mask(*galaxy.shape,(300,520),radius=100)
}

mask[masks.get(galaxy.name,(slice(-1,0),slice(-1,0)))] = True

#img = galaxy.OIII5006_DAP.copy()
img = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.6,1,0.6],percentile=[95,99.,95])
img[mask,...] = (1,1,1)

fig = plt.figure(figsize=(8,8))
ax  = fig.add_subplot()

#norm = simple_norm(galaxy.OIII5006_DAP,clip=False,max_percent=95)
ax.imshow(img,origin='lower')

## Source Detection

In [ ]:
from photutils import DAOStarFinder            # DAOFIND routine to detect sources
from photutils import IRAFStarFinder           # IRAF starfind routine to detect star

from pymuse.detection import detect_unresolved_sources

In [ ]:
with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
    parameters = json.load(json_file)

setattr(galaxy,'aperturesize',parameters[galaxy.name]['aperturesize'])
setattr(galaxy,'binsize',parameters[galaxy.name]['binsize'])
setattr(galaxy,'mu',parameters[galaxy.name]['mu'])
setattr(galaxy,'alpha',parameters[galaxy.name]['power_index'])
setattr(galaxy,'completeness_limit',parameters[galaxy.name]['completeness_limit'])
setattr(galaxy,'roundness',parameters[galaxy.name]['roundness'])
setattr(galaxy,'sharplo',parameters[galaxy.name]['sharplo'])
setattr(galaxy,'sharphi',parameters[galaxy.name]['sharphi'])
setattr(galaxy,'threshold',parameters[galaxy.name]['threshold'])
setattr(galaxy,'zoomin',parameters[galaxy.name]['zoomin'])


threshold = galaxy.threshold
oversize  = 1.

sharplo   = 0.0 #galaxy.sharplo
sharphi   = 1.0 #galaxy.sharphi
roundness = 1.0 #galaxy.roundness

print(f'threshold: {threshold}\nround:     {roundness}\nsharplo:   {sharplo}\nsharphi:   {sharphi}')

sources = detect_unresolved_sources(galaxy,
                                    'OIII5006_DAP',
                                    StarFinder=DAOStarFinder,
                                    threshold=threshold,
                                    exclude_region=mask,
                                    oversize=oversize,
                                    roundlo=-roundness,
                                    roundhi=roundness,
                                    sharplo=sharplo,
                                    sharphi=sharphi,
                                    exclude_border=True,
                                    save=False)

## Completeness limit

In [ ]:
from pymuse.detection import completeness_limit

In [ ]:
with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
    parameters = json.load(json_file)
    
setattr(galaxy,'binsize',parameters[galaxy.name]['binsize'])
setattr(galaxy,'mu',parameters[galaxy.name]['mu'])
setattr(galaxy,'alpha',parameters[galaxy.name]['power_index'])
setattr(galaxy,'completeness_limit',parameters[galaxy.name]['completeness_limit'])
setattr(galaxy,'roundness',parameters[galaxy.name]['roundness'])
setattr(galaxy,'sharplo',parameters[galaxy.name]['sharplo'])
setattr(galaxy,'sharphi',parameters[galaxy.name]['sharphi'])
setattr(galaxy,'threshold',parameters[galaxy.name]['threshold'])
setattr(galaxy,'zoomin',parameters[galaxy.name]['zoomin'])

roundness = galaxy.roundness
sharplo   = galaxy.sharplo
sharphi   = galaxy.sharphi

print(f'for {galaxy.name}')
mock_sources = completeness_limit(
                    galaxy,
                    'OIII5006_DAP',
                    DAOStarFinder,
                    threshold=threshold,
                    iterations=5,
                    stars_per_mag=50,
                    exclude_region=mask,
                    oversize=oversize,
                    roundlo=-roundness,
                    roundhi=roundness,
                    sharplo=sharplo,
                    sharphi=sharphi,
                    exclude_border=True,
                    plot=False
                                 )

## Flux measurement

In the previous step we detected potential PN candidates by their [OIII] emission. This means we know their position but lack exact flux measurments. In this section we measure the flux of the identified objects in different emission lines that are used in later steps. 

In [ ]:
from pymuse.photometry import measure_flux, light_in_moffat, growth_curve

In [ ]:
Rv  = 3.1
Ebv = 0.062
aperture_size = 1.7 #galaxy.aperturesize

flux = measure_flux(galaxy,
                    sources,
                    alpha=galaxy.alpha,
                    Rv=Rv,
                    Ebv=Ebv,
                    extinction='MW',
                    background='local',
                    aperture_size=aperture_size)

# calculate astronomical coordinates for comparison

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

## Emission line diagnostics    

In [ ]:
from pymuse.analyse import emission_line_diagnostics

#galaxy.completeness_limit = 28
#galaxy.mu = 31.29

print(f'emission line diagnostics for {galaxy.name}')
print(f'mu={galaxy.mu:.2f}, cl={galaxy.completeness_limit}')
tbl = emission_line_diagnostics(flux,galaxy.mu,galaxy.completeness_limit) 

tbl['sharp'] = sources['sharpness']
tbl['round'] = sources['roundness2']

### Save catalogue to file

In [ ]:
from pymuse.io import write_LaTeX

filename = basedir / 'data' / 'catalogues' 
write_LaTeX(tbl[c_shape &  c_detec & c_limit],galaxy,filename)

In [ ]:
# this saves the entire table  
tbl_out = tbl[c_shape & (tbl['type']!='NaN') & c_detec]
skycoord = SkyCoord.from_pixel(tbl_out['x'],tbl_out['y'],galaxy.wcs)
tbl_out['RaDec'] = skycoord.to_string(style='hmsdms',precision=2)

for col in tbl_out.colnames:
    if col not in ['id','RaDec','type','SNRorPN']:
        if not col.endswith('detection'):
            tbl_out[col].info.format = '%.3f' 
            
filename = basedir / 'data' / 'catalogues' / f'{galaxy.name}_nebulae.txt'

with open(filename,'w',newline='\n') as f:
    ascii.write(tbl_out,f,format='fixed_width',delimiter='\t',overwrite=True)
print(f'{len(tbl_out)} objects saved to ' + str(filename))   

### Visualize the result of the classification

In [ ]:
from pymuse.plot.pnlf import plot_emission_line_ratio

for t in ['PN','SNR','HII']:
    print(f"{t}: v_sig = {np.nanmean(tbl[(tbl['type']==t) & (tbl['mOIII']<galaxy.completeness_limit)]['v_SIGMA']):.2f}")

filename = basedir / 'reports' / f'{galaxy.name}_emission_line'
plot_emission_line_ratio(tbl[c_shape & c_detec & c_limit],galaxy.mu,completeness=galaxy.completeness_limit,filename=filename)


In [ ]:
from pymuse.plot.classification import classification_map
#parameters[galaxy.name]['zoomin'] = [180,570,250]
#del parameters[galaxy.name]['zoomin']
print(galaxy.name)
filename = basedir / 'reports' / f'{galaxy.name}_detections_classification.pdf'
classification_map(galaxy,parameters,tbl[c_shape & c_detec & c_limit],filename)

## Planetary nebula luminosity function

NGC1365,NGC1566,NGC2835,NGC4254 have overluminous sources

{'IC5332'  : 9,
 'NGC628'  : 9.84,
 'NGC3351' : 9.96,
 'NGC3627' : 11.07,
 'NGC5068' : 5.16,
 
 'NGC1365' : 18.7,
 'NGC2835' : 12.22,
 
 'NGC1512' : 11.6,
 'NGC1566' : 17.69,
 'NGC4254' : 13.1,
 'NGC4535' : 15.77
}

In [ ]:
def prior(mu):
    mu0 = 29.91
    std = 0.13
    
    return 1 / (std*np.sqrt(2*np.pi)) * np.exp(-(mu-mu0)**2 / (2*std**2))

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf
from pymuse.plot.pnlf import plot_pnlf
from pymuse.auxiliary import uncertainties

binsize= 0.4# galaxy.binsize
cut = 26.5
#galaxy.completeness_limit = 28
slow  = 0.2 #galaxy.sharplo  #2 / 10
shigh = 0.9 #galaxy.sharphi #9 / 10
r     = 0.9 #galaxy.roundness #9 / 10

# table contains all detected objects. here we mask all undesired objects.
c_shape = ((tbl['sharp']>slow) & (tbl['sharp']<shigh) & (np.abs(tbl['round'])<r))
c_PN    = (tbl['type']=='PN')
c_SNR   = (tbl['SNRorPN'] & (tbl['type']=='SNR'))
c_AV    = ((tbl['Av']<0.4) | np.isnan(tbl['Av']))
c_cut   = (cut<tbl['mOIII'])
c_detec = tbl['OIII5006_detection'] #| tbl['HA6562_detection'] 
c_limit = (tbl['mOIII']<galaxy.completeness_limit)

criteria = c_shape & c_cut & (c_PN|c_SNR) & c_detec 
data = tbl[np.where(criteria & c_limit)]['mOIII']
err = tbl[np.where(criteria & c_limit)]['dmOIII']

print(f'analysing {galaxy.name}')
print(f'completeness limit = {galaxy.completeness_limit}, binsize = {binsize}')
fitter = MaximumLikelihood1D(pnlf,data,mhigh=galaxy.completeness_limit,Mmax=-4.47)
galaxy.mu,dp,dm = fitter([28])
print('{:.2f} + {:.2f} - {:.2f}'.format(*uncertainties(galaxy.mu,dp,dm)))



#Plot PNLF
meta = {'Subject':f'slow={slow},shigh={shigh},r={r}','Title':f'PNLF for {galaxy.name}','Author':'FS'}
filename = basedir / 'reports' / f'{galaxy.name}_PNLF'
axes = plot_pnlf(tbl[criteria]['mOIII'],
                 galaxy.mu,
                 galaxy.completeness_limit,
                 binsize=binsize,
                 mhigh=30,
                 filename=filename,
                 color=tab10[0])


### Compare to literature

In [ ]:
from pymuse.plot.pnlf import compare_distances
print(galaxy.name)
filename = basedir / 'reports' / f'{galaxy.name}_distances'
compare_distances(galaxy.name,galaxy.mu,dp,dm,filename=None)

### With and without SNR

In [ ]:
from pymuse.analyse import MaximumLikelihood1D, PNLF, pnlf
from pymuse.plot.pnlf import plot_pnlf

criteria1 = c_shape & c_cut & (c_PN) & c_detec #& c_AV
data1 = tbl[np.where(c_limit & criteria1)]['mOIII']
err1  = tbl[np.where(c_limit & criteria1)]['dmOIII']

criteria2 = c_shape & c_cut & (c_PN|c_SNR)  & c_detec# & c_AV
data2 = tbl[np.where(c_limit & criteria2)]['mOIII']
err2  = tbl[np.where(c_limit & criteria2)]['dmOIII']

print(f'{galaxy.name}: literature {galaxy.mu:.3f}')
print(f'completeness limit = {galaxy.completeness_limit}')

fitter = MaximumLikelihood1D(pnlf,data1,mhigh=galaxy.completeness_limit)
mu1,dp1,dm1 = fitter([28])
fitter = MaximumLikelihood1D(pnlf,data2,mhigh=galaxy.completeness_limit)
mu2,dp2,dm2 = fitter([28])

print(f'plotting result for {galaxy.name} (binsize={binsize})')
filename = basedir / 'reports' / f'{galaxy.name}_PNLF_with_SNR'
axes = plot_pnlf(tbl[criteria1]['mOIII'],mu1,galaxy.completeness_limit,binsize=binsize,mhigh=30,color=tab10[0])
axes = plot_pnlf(tbl[criteria2]['mOIII'],mu2,galaxy.completeness_limit,binsize=binsize,mhigh=30,filename=filename,color='grey',alpha=0.7,axes=axes)
plt.show()

### Look at brightest objects

In [ ]:
from pymuse.plot.plot import single_cutout
from pymuse.photometry import growth_curve

tmp = tbl[criteria].copy()

tmp['SkyCoord'] = SkyCoord.from_pixel(tmp['x'],tmp['y'],galaxy.wcs)
tmp['RaDec'] = tmp['SkyCoord'].to_string(style='hmsdms',precision=2)
# smaller 0 is HII region
tmp['HII'] = tmp['R'] + tmp['dR'] + 0.37*tmp['MOIII'] + 1.16 
if 'SkyCoord' in tmp.colnames:
    del tmp['SkyCoord']

# 'x','y','R','dR','MOIII'
columns = ['id','type','sharp','round','Av','mOIII','HII','RaDec','fwhm']
for col in columns[2:-2]:
    tmp[col].info.format = '%.3f'  

tmp.sort('mOIII')
print(tmp[0:10][columns])

In [ ]:
idx = 191   

s = 32
row = tmp[tmp['id']==idx]
ax1,ax2,ax3 = single_cutout(galaxy,row['x'],row['y'],size=s,percentile=95,aperture_size=aperture_size*row['fwhm'][0]/2)


fit,pcov = growth_curve(galaxy.OIII5006,row['x'][0],row['y'][0],'moffat',rmax=4*row['fwhm'][0],plot=True)
alpha, gamma = fit[0], fit[1]
fwhm = 2*gamma * np.sqrt(2**(1/alpha)-1)
print(f'measured: {fwhm:.2f} vs reported:{row["fwhm"][0]:.2f}')

In [ ]:
def point_like(data,sources):
    
    shape = np.zeros(len(sources),dtype=bool)
    
    for i,row in enumerate(sources):
    
        try:
            fit,pcov = growth_curve(data,row['x'],row['y'],'moffat',rmax=6*row['fwhm'],plot=True)
            #err = np.sqrt(np.diag(pcov))
        except:
            continue
        
        alpha, gamma = fit[0], fit[1]
        fwhm = 2*gamma * np.sqrt(2**(1/alpha)-1)
        
        if np.abs(fwhm-row['fwhm'])/ row['fwhm'] <1:
            shape[i] = True
        
    return shape
    #print(err)
    #print(f'measured: {fwhm:.2f} vs reported:{row["fwhm"][0]:.2f}')
          
shape = point_like(data,sources)

## Compare to existing Studies

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.table import vstack

from pymuse.load_references import NGC628, \
                                   pn_NGC628_kreckel, \
                                   snr_NGC628_kreckel, \
                                   pn_NGC628_herrmann, \
                                   NGC628_kreckel, \
                                   pn_NGC5068_herrmann, \
                                   pn_NGC3351_ciardullo, \
                                   pn_NGC3627_ciardullo


def get_fwhm(x,y):
    try:
        return galaxy.PSF[int(y),int(x)]
    except:
        return 0

for table in [NGC628,pn_NGC628_kreckel,snr_NGC628_kreckel,NGC628_kreckel,
              pn_NGC628_herrmann,pn_NGC5068_herrmann,pn_NGC3351_ciardullo,pn_NGC3627_ciardullo]:
    table['x'],table['y']= table['SkyCoord'].to_pixel(wcs=galaxy.wcs)
    table['fwhm'] = np.array([get_fwhm(x,y) for x,y in zip(table['x'],table['y'])])


exclude objects that lie outside our field of view

In [ ]:
matchcoord = NGC628 

catalogcoord = tbl[tbl['mOIII']<galaxy.completeness_limit]

matchcoord['in_frame'] = False
y_dim,x_dim = galaxy.shape

for row in matchcoord:
    x,y = row['x'], row['y']    
    if 0<=int(x)<x_dim and 0<=int(y)<y_dim:
        if not np.isnan(galaxy.PSF[int(y),int(x)]):
            row['in_frame'] = True
           
print(f"{np.sum(~matchcoord['in_frame'])} objects outside of our field of view")
matchcoord   = matchcoord[matchcoord['in_frame'] & (matchcoord['mOIII']<27.5)]

plot the detections from the paper and our own detections

In [ ]:
fig = plt.figure(figsize=(single_column,single_column))
ax1 = fig.add_subplot(111,projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax1.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

ax1.scatter(matchcoord['x'],matchcoord['y'],marker='o',s=6,lw=0.2,edgecolor='tab:red',facecolors='none')
ax1.scatter(catalogcoord['x'],catalogcoord['y'],marker='o',s=6,lw=0.2,edgecolor='tab:orange',facecolors='none')

for row in matchcoord:
    txt,x,y = row['ID'], row['x']+5, row['y']    
    
    ax1.annotate(txt, (x, y),fontsize=4,color='tab:red')

plt.savefig(basedir / 'reports' / f'{galaxy.name}_PN_position_comparison.pdf',dpi=600)

see how many match within 1"

In [ ]:
tolerance = '0.8"'
ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle[angle.__lt__(Angle(tolerance))].mean().to_string(u.arcsec,decimal=True)}')

### Compare H$\alpha$ fluxes

In [ ]:
tolerance = '0.8"'

ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],SkyCoord.from_pixel(catalogcoord['x'],catalogcoord['y'],galaxy.wcs))
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle[angle.__lt__(Angle(tolerance))].mean().to_string(u.arcsec,decimal=True)}')

In [ ]:
def compare_OIII_fluxes():
    mpl.use('pgf')
    plt.style.use('TeX.mplstyle')
    
    mpl.rcParams['pgf.preamble'] = [r'\usepackage[hidelinks]{hyperref}', ]
    
    matchcoord['mOIII_measured'] = catalogcoord[ID]['mOIII']
    matchcoord['dmOIII_measured'] = catalogcoord[ID]['dmOIII']
    
    crit = angle.__lt__(Angle("1s"))

    fig,ax = plt.subplots(figsize=(single_column,single_column))

    color=tab10[0]
    for s in ['Kreckel PN','Kreckel SNR','Herrmann PN']:
        color = next(ax._get_lines.prop_cycler)['color']
        tmp = matchcoord[(matchcoord['source']==s) & crit]
        plt.errorbar(tmp['mOIII'],tmp['mOIII_measured'],
                     yerr = tmp['dmOIII_measured'],
                     marker='o',ms=2,ls='none',mec=color,mfc=color,ecolor=color,label=s)

    #plt.errorbar(matchcoord[crit]['mOIII'],matchcoord[crit]['mOIII_measured'],
    #             yerr = matchcoord[crit]['dmOIII_measured'],
    #             marker='o',ms=4,ls='none',mec=color,mfc=color,ecolor=color,label=s)

        
    base_url = 'https://ui.adsabs.harvard.edu/abs/'
    link = f'\href{{{base_url + matchcoord.meta["bibcode"]}}}{{{matchcoord.meta["reference"]}}}'

    xmin = np.floor(2*np.min(matchcoord['mOIII']))/2
    xmax = np.ceil(2*np.max(matchcoord['mOIII']))/2
    ymin = np.floor(2*np.min(matchcoord['mOIII_measured']))/2
    ymax = np.ceil(2*np.max(matchcoord['mOIII_measured']))/2
    
    plt.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
    plt.plot([xmin,xmax],[xmin-0.5,xmax-0.5],color='gray',lw=0.5,ls='--')
    plt.plot([xmin,xmax],[xmin+0.5,xmax+0.5],color='gray',lw=0.5,ls='--')
    #ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$' + f' {link}')
    ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ existing studies')
    ax.set_ylabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ this work')
    plt.legend()
    
    #plt.savefig(basedir / 'reports' / f'flux_comparison_OIII.pdf',dpi=600)
    plt.show()
    
compare_OIII_fluxes()

In [ ]:
def compare_HA_fluxes():
    
    mpl.use('pgf')
    plt.style.use('TeX.mplstyle')
    
    mpl.rcParams['pgf.preamble'] = [r'\usepackage[hidelinks]{hyperref}', ]
    
    catalogcoord['R2'] = catalogcoord['OIII5006'] / (catalogcoord['HA6562']+catalogcoord['NII6583'])
    catalogcoord['dR2'] = catalogcoord['R2']  * np.sqrt(catalogcoord['OIII5006_err']/catalogcoord['OIII5006_err']**2 + 1/(catalogcoord['HA6562']+catalogcoord['NII6583'])**2 * (catalogcoord['HA6562_err']**2+catalogcoord['NII6583_err']**2) )                                  
    
    matchcoord['R_measured'] = catalogcoord[ID]['R2']
    matchcoord['dR_measured'] = catalogcoord[ID]['dR2']

    crit = angle.__lt__(Angle("1s"))

    fig,ax = plt.subplots(figsize=(single_column,single_column))

    color=tab10[0]
    for s in ['Kreckel PN','Kreckel SNR','Herrmann PN']:
        color = next(ax._get_lines.prop_cycler)['color']
        tmp = matchcoord[(matchcoord['source']==s) & crit]
        print(f"{s}: {np.sum(tmp['R'] > tmp['R_measured']) / len(tmp) * 100:.2f} % under")
        plt.errorbar(tmp['R'],tmp['R_measured'],
                     #xerr = tmp['dR'],
                     #yerr = tmp['dR_measured'],
                     marker='o',ms=2,ls='none',mec=color,mfc=color,ecolor=color,label=s)

    xmin,xmax = 0,10
    ymin,ymax = 0,10
    
    plt.plot([xmin,xmax],[xmin,xmax],color='black',lw=0.4)
    plt.plot([xmin,xmax],[xmin-0.5,xmax-0.5],color='gray',lw=0.5,ls='--')
    plt.plot([xmin,xmax],[xmin+0.5,xmax+0.5],color='gray',lw=0.5,ls='--')
    ax.set_xlim([0,10])
    ax.set_ylim([0,10])
    #ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$' + f' {link}')
    ax.set_xlabel(r'[OIII]/(H$\alpha$+[NII]) existing studies')
    ax.set_ylabel(r'[OIII]/(H$\alpha$+[NII]) this study')
    plt.legend(loc=1)
    
    
    plt.savefig(basedir / 'reports' / f'flux_comparison_HA.pdf',dpi=600)
    
    plt.show()
    
catalogcoord = tbl[tbl['mOIII']<galaxy.completeness_limit]
compare_HA_fluxes()

### Francesco's Nebula catalogue

In [ ]:
from photutils import CircularAperture

with fits.open(basedir / 'data' / 'external' / 'nebula_catalogue_FS_v01.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)

# (nebula_catalogue['region_size_pixels']<100) &
PNe_candidate = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name) & (nebula_catalogue['PNe_candidate']==1)]
print(f'{len(PNe_candidate)} candidates from FS')

fig = plt.figure(figsize=(7,7))
ax  = fig.add_subplot(projection=galaxy.wcs)
norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

cat = tbl[c_PN & c_limit]

#positions = np.transpose([PNe_candidate['cen_x'],PNe_candidate['cen_y']])
#apertures = CircularAperture(positions, r=6)
ax.scatter(PNe_candidate['cen_x'],PNe_candidate['cen_y'],marker='o',s=5,lw=0.4,edgecolor='red',facecolors='none')
ax.scatter(cat['x'],cat['y'],marker='o',s=5,lw=0.4,edgecolor='blue',facecolors='none')

#apertures.plot(color='red',lw=.2, alpha=1)
plt.savefig(basedir / 'reports' / f'{galaxy.name}_FS_comparison.pdf',dpi=600)

In [ ]:
data = -2.5*np.log10(PNe_candidate['OIII5006_FLUX']*1e-20) - 13.74

fitter = MaximumLikelihood1D(pnlf,
                             data[data<28],
                             mhigh=galaxy.completeness_limit)
mu,dp,dm = fitter([24])
print(f'literature: {galaxy.mu:.2f}')
axes = plot_pnlf(data,
                 mu,
                 galaxy.completeness_limit,
                 binsize=0.4,
                 mhigh=30,
                 color=tab10[0])


### Enrico's Catalogue

In [ ]:
from photutils import CircularAperture

with fits.open(basedir / 'data' / 'external' / 'clumpfind_cat_v02.fits') as hdul:
    nebula_catalogue = Table(hdul[1].data)
nebula_catalogue = nebula_catalogue[(nebula_catalogue['gal_name']==galaxy.name)]

nebula_catalogue.rename_column('cen_x','x')
nebula_catalogue.rename_column('cen_y','y')
nebula_catalogue = nebula_catalogue[~np.isnan(nebula_catalogue['x']) & ~np.isnan(nebula_catalogue['y'])]
nebula_catalogue['fwhm'] = np.array([galaxy.PSF[int(y),int(x)] for x,y in zip(nebula_catalogue['x'],nebula_catalogue['y'])])

#### Use flux measurements from Enrico

In [ ]:
for col in nebula_catalogue.colnames:
    if col.endswith('_FLUX'):
        nebula_catalogue.rename_column(col,col[:-5])
    if col.endswith('_FLUX_ERR'):
        nebula_catalogue.rename_column(col,col[:-9]+'_err')
        
nebula_catalogue['mOIII'] = -2.5*np.log10(nebula_catalogue['OIII5006']*1e-20) - 13.74
nebula_catalogue['dmOIII'] = np.abs( 2.5/np.log(10) * nebula_catalogue['OIII5006_err'] / nebula_catalogue['OIII5006'])

nebula_catalogue = emission_line_diagnostics(nebula_catalogue,galaxy.mu,galaxy.completeness_limit) 
pn_candidates = nebula_catalogue[nebula_catalogue['type']=='PN']

#### Measure flux with background subtraction

In [ ]:
flux = measure_flux(galaxy,
                    nebula_catalogue,
                    alpha=galaxy.alpha,
                    Rv=3.1,
                    Ebv=0.062,
                    extinction='MW',
                    aperture_size=1.5)

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

emd = emission_line_diagnostics(flux,galaxy.mu,galaxy.completeness_limit) 
pn_candidates = emd[(emd['type']=='PN') & (emd['mOIII']<28)]


#### Visualize the result

In [ ]:
print(f'{len(pn_candidates)} nebulae from Enrico')

fig = plt.figure(figsize=(7,7))
ax  = fig.add_subplot(projection=galaxy.wcs)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

cat = tbl[c_PN & c_limit]

#positions = np.transpose([PNe_candidate['cen_x'],PNe_candidate['cen_y']])
#apertures = CircularAperture(positions, r=6)
ax.scatter(pn_candidates['x'],pn_candidates['y'],marker='o',s=4,lw=0.4,edgecolor='tab:orange',facecolors='none')
ax.scatter(cat['x'],cat['y'],marker='o',s=6,lw=0.4,edgecolor='tab:blue',facecolors='none')

#apertures.plot(color='red',lw=.2, alpha=1)
plt.savefig(basedir / 'reports' / f'{galaxy.name}_Enrico_comparison.pdf',dpi=600)

In [ ]:
from astropy.coordinates import match_coordinates_sky

matchcoord   = pn_candidates
matchcoord['SkyCoord'] = SkyCoord.from_pixel(matchcoord['x'],matchcoord['y'],galaxy.wcs)
cat['SkyCoord'] = SkyCoord.from_pixel(cat['x'],cat['y'],galaxy.wcs)

tolerance = '2s'
ID, angle, Quantity  = match_coordinates_sky(matchcoord['SkyCoord'],cat['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle[angle.__lt__(Angle(tolerance))].mean().to_string(u.arcsec,decimal=True)}"')

In [ ]:
fitter = MaximumLikelihood1D(pnlf,
                             pn_candidates[(pn_candidates['mOIII']<28) & (pn_candidates['mOIII']>10)]['mOIII'],
                             mhigh=galaxy.completeness_limit)
mu,dp,dm = fitter([24])
print(f'literature: {galaxy.mu:.2f}')
axes = plot_pnlf(pn_candidates['mOIII'],
                 mu,
                 galaxy.completeness_limit,
                 binsize=0.4,
                 mhigh=30,
                 color=tab10[0])


## Distance in parsec

the measured distances are in the form of the distance modulus $\mu = m-M$ which is the difference between apparent and absolute magnitude. By defintion of the absolte magnitude, we can convert this number into a distance in pc
$$
d = 10^{\frac{\mu}{5}+1} = 10 \cdot \exp\left( \ln 10 \frac{\mu}{5} \right) \\
\delta d = \frac{\ln 10}{5} 10 \exp\left( \ln 10 \frac{\mu}{5} \right) \delta \mu = 0.2 \ln 10 \; d \; \delta \mu
$$

In [ ]:
def distance_modulus_to_parsec(mu,mu_err=np.array([])):
    
    d = 10 * np.exp(np.log(10)*mu/5)
    if len(mu_err) > 0:
        d_err = 0.2 * np.log(10) * d * mu_err
    print(f'd = ({d/1e6:.2f} + {d_err[0]/1e6:.2f} - {d_err[1]/1e6:.2f}) Mpc')
    
    return d, d_err

d,d_err = distance_modulus_to_parsec(30.033,np.array([0.014,0.015]))

## Dust extinction etc.

The V-band extinction $A_V$ and the color excess $E(B-V)=(B-V)_\text{obs} - (B-V)_\text{int}$ are related via the selective extinction
$$
A_V = R_V E(B-V)
$$
The extinction at wavelength $\lambda$ can be obtained with the extinction curve
$$
k(\lambda) = \frac{A_\lambda}{E(B-V)}\quad \rightarrow\quad A_\lambda = k(\lambda) E(B-V)
$$
or 
$$
k(\lambda) = \frac{A_\lambda}{A_V} R_V
$$
The color excess can be calculated if the intrinsic ratio of two lines is known
$$
\begin{align}
E(B-V) &= \frac{E(F_2-F_1)}{k(\lambda_2) -k(\lambda_1)} \\
&=\frac{2.5}{k(\lambda_2) - k(\lambda_1)} \log_{10} \left[ \frac{(F_1 / F_2)_\text{obs}}{(F_1 / F_2)_\text{int}} \right] 
\end{align}
$$



`dust_extinction.evaluate` returns $A_\lambda / A_V$, hence we need to multiply with $R_V$ to get $k(\lambda)$


`dust_extinction.extinguish` returns the fractional extinction $f_\lambda$. To get $A_\lambda$ we need to $-2.5\log_{10} f_\lambda$

$$
E(B-V)_\text{star} = 0.44 E(B-V)_\text{nebula}
$$

### Attenuation vs extinction

extinction only considers one ray of light. Therefor it will only decrease the intensity. For attenuation, a larger area and a bundle of light rays is considered. It is possible that light is scattered from one line of sight into another one.

In [ ]:
from dust_extinction.parameter_averages import CCM89, F99

In [ ]:
Rv=3.1

ext_model = CCM89(Rv=Rv)
k = lambda lam: ext_model.evaluate(lam*u.angstrom,Rv) * Rv

def calculateEbv(F1,F2,lam1,lam2,Rint):
    '''
    
    Rint : float
        intrinsic flux ratio flux1/flux2
    '''
        
    with np.errstate(divide='ignore', invalid='ignore'):
        Ebv = 2.5 / (k(lam2)-k(lam1)) * np.log10(F1/F2/Rint)
    
    Ebv[~np.isfinite(Ebv)] = np.nan
    
    return Ebv

Ebv = calculateEbv(galaxy.HA6562,galaxy.HB4861,6562,4861,2.86)
Ebv[(galaxy.HB4861 / galaxy.HB4861_err < 5) & (galaxy.HA6562 / galaxy.HA6562_err < 15)] = np.nan

Av = Ebv * Rv


norm = simple_norm(Av,'linear',clip=False,min_cut=-5,max_cut=5)
im = plt.imshow(Av,origin='lower',norm=norm)
plt.colorbar(im)
plt.show()

In [ ]:
row = sources[1]

Av = np.array([galaxy.Av[int(row['x']),int(row['y'])] for row in sources])

In [ ]:
ext_model.extinguish(5007*u.angstrom,Av=Av)

In [ ]:
x*ext_model.evaluate(5007*u.angstrom,Rv=3.1)

In [ ]:
x=0.
10**(-0.4*x*k(5007)/Rv)

In [ ]:
ext_model.extinguish(5007*u.angstrom,Av=x)

In [ ]:
external = Path('g:/Archive')

with fits.open(external / 'MUSE' / 'AUXILIARY' / 'AVmaps' / 'fits' / f'{galaxy.name}_AV_caseB_negnan_conv_broad_Ha15_Hb5.fits') as hdul:
    AVmap = hdul[0].data

In [ ]:
#norm = simple_norm(AVmap,'linear',clip=False,percent=99.9)
im = plt.imshow(AVmap,origin='lower',norm=norm)
plt.colorbar(im)
plt.show()

In [ ]:
dif = np.abs((AVmap-Av)/Av)
norm = simple_norm(dif,'linear',clip=False,min_cut=0,max_cut=1)
fig = plt.figure(figsize=(10,10))
im = plt.imshow(dif,origin='lower',norm=norm)
plt.colorbar(im)
plt.show()

In [ ]:
box_size = 4
xmax, ymax = Av.shape

out = np.zeros(Av.shape)

for i in range(xmax):
    for j in range(ymax):
        mask = slice(max(0,i-box_size),min(i+box_size,xmax)),slice(max(0,j-box_size),min(j+box_size,ymax))        
        out[i,j] = np.nanmean(Av[mask])

In [ ]:
np.nanmean(out[(out<np.inf) & (out>-np.inf)])

In [ ]:
lam = 5007
Alam =  ext_model.evaluate(lam*u.angstrom,Rv) * AVmap

In [ ]:
def presentation(galaxy,parameters,filename):
    '''
    Plot data in galaxy for different lines and overplot the position
    of detected sources
    '''

    if "zoomin" in parameters[galaxy.name]:
        x,y,width = 380,400,200
        height = width * galaxy.shape[0] / galaxy.shape[1]

        if x+width > galaxy.shape[1]:
            print(f'cutout is too wide')
            new_width =  galaxy.shape[1] - x
            height *= new_width / width 
            width = new_width

        if y+height > galaxy.shape[0]:
            print(f'cutout is to high')
            new_height =  galaxy.shape[0] - y
            width *= new_height / height
            height = new_height

    else:
        x,y=0,0
        height,width = galaxy.shape

    # ====== define input parameters =============================
    labels=['SII6716','HA6562','OIII5006']
    wcs=galaxy.wcs
    # ============================================================

    from pymuse.plot.plot import create_RGB
    #rgb = create_RGB(galaxy.HA6562,galaxy.OIII5006,galaxy.SII6716,percentile=97)
    rgb = create_RGB(galaxy.HA6562,galaxy.OIII5006_DAP,galaxy.SII6716,weights=[0.8,1,0.9],percentile=[97,97,97])

    table = tbl
    #table = tbl[tbl['mOIII']<galaxy.completeness_limit]

    fig = plt.figure(figsize=(6.974,6.974/2))
    ax1 = fig.add_subplot(121,projection=wcs)
    ax2 = fig.add_subplot(122)

    norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=98)
    ax1.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

    norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=98)
    ax2.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Greens)

    
    
    ax1.add_patch(mpl.patches.Rectangle((x,y),width,height,linewidth=0.3,edgecolor='k',facecolor='none'))
    ax2.set_xlim([x,x+width])
    ax2.set_ylim([y,y+height])

    ax1.set(title=r'O[III]',
            xlabel='R.A. (J2000)',
            ylabel='Dec. (J2000)')

            
    ax2.set_xticks([])
    ax2.set_yticks([])

    # format ticks with wcs
    # https://docs.astropy.org/en/stable/visualization/wcsaxes/ticks_labels_grid.html
    ax1.coords[0].set_ticks(number=3)
    ax1.coords[1].set_ticks(number=4)


    # https://matplotlib.org/3.1.3/api/_as_gen/matplotlib.patches.ConnectionPatch.html
    con = mpl.patches.ConnectionPatch(xyA=((x+width)/galaxy.shape[1],(y+height)/galaxy.shape[0]), xyB=(0,1), 
                                      coordsA="axes fraction", coordsB="axes fraction",
                                      axesA=ax1, axesB=ax2, color="black",linewidth=0.3)
    ax2.add_artist(con)
    con = mpl.patches.ConnectionPatch(xyA=((x+width)/galaxy.shape[1],(y)/galaxy.shape[0]), xyB=(0,0), 
                                      coordsA="axes fraction", coordsB="axes fraction",
                                      axesA=ax1, axesB=ax2, color="black",linewidth=0.3)
    ax2.add_artist(con)

    plt.show()
    #plt.savefig(filename,bbox_inches='tight',dpi=600)

presentation(galaxy,parameters,'test.pdf')

In [ ]:
import yaml

In [ ]:
with open('parameters.yml', 'w') as outfile:
    yaml.dump(parameters, outfile, default_flow_style=False)

In [ ]:
with open("parameters.yml", 'r') as stream:
    parameters_loaded = yaml.safe_load(stream)

In [ ]:
filename = basedir / 'data' / 'external' / 'phangs_sample_table_v1p5.fits'
with fits.open(filename) as hdul:
    sample_table = Table(hdul[1].data)

muse_sample = sample_table[sample_table['survey_muse_status']!='not_in_survey']
